In [71]:
import pandas as pd
import numpy as np

In [72]:
df1 = pd.read_csv('../small1/labeled.csv')
df2 = pd.read_csv('../small2/labeled.csv')
df3 = pd.read_csv('../small3/labeled.csv')

In [73]:
df = pd.concat([df1, df2, df3])

In [74]:
# drop empty titles
df = df[df['title'].notna()]

In [75]:
df = df.reset_index(drop=True)
df

,title,is_clickbait
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1
...,...,...
75017,"To Make Female Hearts Flutter in Iraq, Throw a...",0
75018,"British Liberal Democrat Patsy Calton, 56, die...",0
75019,Drone smartphone app to help heart attack vict...,0
75020,"Netanyahu Urges Pope Benedict, in Israel, to D...",0


In [76]:
from nltk.corpus import stopwords

In [77]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [78]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('', '', string.punctuation))

In [79]:
# find 100 most common words in title column

from collections import Counter

# remove stopwords from title column
df['title_removed'] = df['title'].apply(remove_stopwords)

# remove posesive 's
df['title_removed'] = df['title_removed'].str.replace("'s", "")

# remove declinations
df['title_removed'] = df['title_removed'].str.replace("'ve", " have")
df['title_removed'] = df['title_removed'].str.replace("'re", " are")
df['title_removed'] = df['title_removed'].str.replace("'ll", " will")
df['title_removed'] = df['title_removed'].str.replace("'d", " would")
df['title_removed'] = df['title_removed'].str.replace("n't", " not")

# remove punctuation
df['title_removed'] = df['title_removed'].apply(remove_punctuation)

# remove numbers
df['title_removed'] = df['title_removed'].str.replace('\d+', '')

# count words in title column
title_words = df['title_removed'].str.split(expand=True).stack()
title_words = title_words.value_counts()

In [80]:
title_words[:100]

New        9096
Times      7200
York       6873
Trump      5645
US         2875
           ... 
Say         394
Twitter     393
Could       389
Years       383
5           382
Name: count, Length: 100, dtype: int64